### Theoretical part
1. hoe werkt markdown en doe ik linebreaks
2.

### Experimental part

##### Step 1

In [40]:
import math, random, itertools, numpy as np

In [41]:
def pair_generator():
    """
    A generator that returns pairs of all possible combinations 
    of [N, R, HR] of length 5, with repeated elements.
    """
    pairs = itertools.product(itertools.product(['N', 'R', 'HR'], repeat=5), repeat=2)
    for p in pairs:
        yield p

print('Number of pairs:', len(list(pair_generator())))

print('first 10 pairs:')
pair_gen = pair_generator()
for _ in range(10):
    print(next(pair_gen))

def random_sample(length):
    '''
    Returns a sample pair that
    consists of a production and 
    an experiment list, with as possible
    values {N, R, HR}.
    '''
    values = ['N', 'R', 'HR']
    
    p = [values[random.randint(0, 2)] for _ in range(length)]
    e = [values[random.randint(0, 2)] for _ in range(length)]
    
    return p, e


Number of pairs: 59049
first 10 pairs:
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'N', 'N'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'N', 'R'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'N', 'HR'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'R', 'N'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'R', 'R'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'R', 'HR'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'HR', 'N'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'HR', 'R'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'HR', 'HR'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'R', 'N', 'N'))


(['HR', 'R', 'N', 'HR', 'R'], ['R', 'N', 'R', 'N', 'R'])

In [38]:
# Implement Evaluation Measures

import numpy as np

def binary_precision(p, f=None):
    return np.array([0 if x == 'N' else 1 for x in p]).sum() / len(p)

def ndcg(p, relevance_map):
    dcg = np.array([
        (2 ** relevance_map[x] - 1) / np.log2(r + 1) for r, x in enumerate(p, start=1) 
    ])
    return (dcg / dcg.max()).sum()
    
def err(p, relevance_map):
    P = 1
    E = 0
    for r, v in enumerate(p, start=1):
        R = (2 ** relevance_map[v] - 1) / (2 ** max(relevance_map.values()))
        E += P * R / r
        P *= (1-R)
    return E
    
p, e = random_sample(5)

relevance_map = {
    'N': 0,
    'R': 1,
    'HR': 2
}

print('Precision of production algorithm:\t', binary_precision(p))
print('Precision of experimental algorithm:\t', binary_precision(e))

print('EER of production algorithm:\t\t', err(p, relevance_map))
print('EER of experimental algorithm:\t\t', err(e, relevance_map))

print('NDCG of production algorithm:\t\t', ndcg(p, relevance_map))
print('NDCG of experimental algorithm:\t\t', ndcg(e, relevance_map))

Precision of production algorithm:	 0.8
Precision of experimental algorithm:	 0.8
EER of production algorithm:		 0.425
EER of experimental algorithm:		 0.36875
NDCG of production algorithm:		 2.46037884801
NDCG of experimental algorithm:		 2.31752936531


In [29]:
# Calculate the 𝛥measure

def avg_delta(n, eval_function):
    return sum(
        [eval_function(x[0], relevance_map) - eval_function(x[1], relevance_map) for 
         x in [random_sample(5) for _ in range(n)]]
    ) 

    
print(avg_delta(50, binary_precision))
print(avg_delta(50, eer))
print(avg_delta(50, ndcg))

1.6
1.3791666666666655
nan


/Users/joerisleegers/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [37]:
def teamdraft_interleaving(p, e):
    team_a = set()
    team_b = set()
    i = []
    
    while (len(p) + len(e)) > 0:
        if len(team_a) < len(team_b) or (len(team_a) == len(team_b) and random.random() > 0.5):
            k = p.pop(0)
            i.append(k)
            team_a.add(k)
        else:
            k = e.pop(0)
            i.append(k)
            team_b.add(k)
    return i, team_a, team_b

teamdraft_interleaving(p, e)

IndexError: pop from empty list

In [35]:
random.random()

0.576893042361473